In [1]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

## Autó sebesség szabályozása

A sebesség karakterisztikája egy autónak az alábbiképp modellezhető:

$$
\begin{cases}
\dot{x}=\underbrace{\begin{bmatrix}-\frac{b}{m}&\frac{\tau_{\text{max}}\eta}{mr}\\0&-\frac{1}{2}\end{bmatrix}}_{A}x+\underbrace{\begin{bmatrix}0\\\frac{1}{2}\end{bmatrix}}_{B}t_r \\
y = \underbrace{\begin{bmatrix}3.6&0\end{bmatrix}}_{C}x
\end{cases}
$$

ahol $y$ az autó sebessége (km/h), az autó tömege $m$ = 1000 kg, a maximális motor nyomaték $\tau_{\text{max}}$ = 150 Nm, a kerék sugara $r$ = 25 cm (az áttétel konstans $\eta$ 4:1 motor:kerék), a légellenállás $b$ = 60 Ns/m és $\tau_{\%}$ a nyomatékvezérlő jel (0 és 1 közötti érték). A rendszer irányítható és megfigyelhető.

A cél egy olyan szabályozó rendszer tervezése, ami az autó sebességét befolyásolja a motor nyomaték változtatásával az alábbi aramétereknek megfelelően:
- nincs maradó hiba egységugrás bemenet esetében,
- nincs túllövés, vagy legfeljebb 1%,
- a beállási idő 5%-os határon belülre kevesebb, mint 4 másodperc.

### Regulátor tervezése
Fontos megjegyezni, hogy nem lehetséges a beállási időt minden sebesség referencia esetében teljesíteni. Ez egyszerűen belátható fizikai megközelítésből: a légellenállás jelentősen nagyobb magas sebességek esetében, így az autó nem tud úgy gyorsulni magas sebességeken, mint alacsonyokon. Továbbá, egyszerűen bemutatható, hogy még elhanyagolható légellenállás ($b=0$) esetében is, a maximális elérhető sebesség változás maximális nyomaték mellett 5 másodperc alatt 43.2 km/h.
Ezáltal a szabályozót úgy hozzuk létre, hogy az első két szempontot priorizálja, és próbáljon minél gyorsabb lenni.

#### Állapot visszacsatolás tervezése

Nulla maradó hiba eléréséhez a rendszert kiegészítjük egy új $x_3 = v-v_{\text{ref}}$ állapottal, ahol $v_{\text{ref}}$ lesz a referencia bemenet km/h-ban. A kiegészített rendszer állapotegyenletei:

$$
\begin{cases}
\dot{x_a}=\underbrace{\begin{bmatrix}-\frac{b}{m}&\frac{\tau_{\text{max}}\eta}{mr}&0\\0&-\frac{1}{2}&0\\3.6&0&0\end{bmatrix}}_{A_a}x_a+\underbrace{\begin{bmatrix}0\\\frac{1}{2}\\0\end{bmatrix}}_{B_a}t_r+\underbrace{\begin{bmatrix}0\\0\\-1\end{bmatrix}}_{B_{\text{ref}}}v_{\text{ref}} \\
y_a = \underbrace{\begin{bmatrix}3.6&0&0\\0&0&1\end{bmatrix}}_{C_a}x_a
\end{cases}
$$

Vegyük észre, hogy a rendszer irányítható marad.

Egy lehetséges megoldás az $A-BK$ mátrix sajátértékeinek $-0.06$, $-2.43-2.41i$ és $-2.43+2.41i$ értékekre módosítása a $K_a=\begin{bmatrix}9.75&8.73&0.15\end{bmatrix}$ erősítés segítségével.

#### Megfigyelő tervezése

A megfigyelő létrehozható az eredeti rendszer alapján, $x_1$ és $x_2$ becslésére. Az ($A-LC$) mátrix pólusai a megfelelően gyors konvergáláshoz $-10$rad/s-be kerülnek az $L=\begin{bmatrix}5.4&10.45\end{bmatrix}^T$ erősítés segítségével.

A rendszer és a szabályozó szimulációja lentebb látható. Lehetőség van a bemenet telítésének szimulációjára is.

In [2]:
%matplotlib inline
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell

Aa = numpy.matrix([[-3/50,2.4,0],[0,-0.5,0],[3.6,0,0]])
Ba = numpy.matrix('0; 0.5; 0')
A = numpy.matrix([[-3/50, 2.4],[0,-0.5]])
B = numpy.matrix('0; 0.5')
C = numpy.matrix('3.6 0')
X0 = numpy.matrix('0.; 0.')
K = numpy.matrix([9.75,8.73,0.15])
L = numpy.matrix([[5.4],[10.45]])

# Aw = matrixWidget(3,3)
# Aw.setM(A)
# Bw = matrixWidget(3,1)
# Bw.setM(B)
# Cw = matrixWidget(1,3)
# Cw.setM(C)
X0w = matrixWidget(2,1)
X0w.setM(X0)
Kw = matrixWidget(1,3)
Kw.setM(K)
Lw = matrixWidget(2,1)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-0.06])) 
eig2c.setM(numpy.matrix([[-2.43],[-2.41]]))
eig3c.setM(numpy.matrix([-0.06]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig1o.setM(numpy.matrix([-10.])) 
eig2o.setM(numpy.matrix([[-10.],[0.]]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Vizsgálat',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Vizsgálat',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['K és L megadása', 'Sajátértékek megadása'],
    value= 'K és L megadása',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= ['0 komplex sajátérték', '2 komplex sajátérték'],
    value= '2 komplex sajátérték',
    description='Komplex sajátértékek:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impulzus', 'egységugrás', 'szinusz', 'négyszögjel'],
    value='egységugrás',
    description='Referencia típusa:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=30.,
    min=1.,
    max=150.0,
    step=1,
    description='Referencia:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=30,
    min=1,
    max=80,
    step=1,
    description='Periódus: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
time = widgets.BoundedIntText(
    value=120,
    min=1,
    max=200,
    step=1,
    description='Idő:',
    disabled=False
)
sim = widgets.Checkbox(
    value=False,
    description='Telítés',
    disabled=False,
    indent=False
)

In [5]:
# Support functions

def eigen_choice(sele):
    if sele == '0 komplex sajátérték':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig = 0
    if sele == '2 komplex sajátérték':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig = 2
    return eig

def method_choice(selm):
    if selm == 'K és L megadása':
        method = 1
        sele.disabled = True
    if selm == 'Sajátértékek megadása':
        method = 2
        sele.disabled = False
    return method

def simulation(sys_G, sys_K, sys_int, x0, x0k, time, Ts, ref, lim):
    '''Function that simulate a siso feedback system with a saturation in the actuator and integral action.
       Input to sys_K [u_sys_G; y_sys_G].
       sys_int must have k_3 as gain in output and B = [1, -1] associated to y and ref.'''
    if len(time) != len(ref):
        print('Hiba: T és ref azonos méretű kell legyen!')
        return

    # Initialization and preparation...
    K_temp = control.append(sys_K,sys_int)
    K = sss(K_temp.A, 
            K_temp.B*sym.Matrix([[1,0,0],[0,1,0],[0,1,0],[0,0,1]]), 
            sym.Matrix([1,1]).T*K_temp.C, 
            sym.Matrix([1,1]).T*K_temp.D*sym.Matrix([[1,0,0],[0,1,0],[0,1,0],[0,0,1]]))
    sys_temp = control.append(sys_G,K)
    sys_temp1 = sss(sys_temp.A, 
                    sys_temp.B*sym.Matrix([[1,0,0],[1,0,0],[0,1,0],[0,0,1]]),
                    sys_temp.C,
                    sys_temp.D*sym.Matrix([[1,0,0],[1,0,0],[0,1,0],[0,0,1]]))
    #sys = control.connect(sys_temp1,[[2,1]],[1,3],[1,2])
    sys = sss(sys_temp1.A+sys_temp1.B[:,1]*sys_temp1.C[0,:],
              sym.Matrix(sys_temp1.B[:,0]).row_join(sym.Matrix(sys_temp1.B[:,2])),
              sys_temp1.C,
              [[0,0],[0,0]])
    sys_d = control.c2d(sys,Ts)
    A, B, C, D = sys_d.A, sys_d.B, sys_d.C, sys_d.D

    # Initializing system's variables
    x = [x0.col_join(x0k).col_join(sym.Matrix([0])) for t in range(0,len(time))]
    yinit = C*x[0]
    y = [yinit for t in range(0,len(time))]
    Input = [sym.Matrix([yinit[1],ref[t]]) for t in range(0,len(time))]

    for i in range(1,len(time)):
        Input[i-1][0] = -y[i-1][1]
        if Input[i-1][0] > lim:
            Input[i-1][0] = lim
        elif Input[i-1][0] < 0:
            Input[i-1][0] = 0.
    
        x[i] = A*x[i-1] + B*Input[i-1]
        y[i] = C*x[i-1] + D*Input[i-1]
        
    if Input[-1][0] > lim:
            Input[-1][0] = lim
    elif Input[-1][0] < 0:
            Input[-1][0] = 0.
            
    y  = [y[t][0] for t in range(len(y))]
    U  = [Input[t][0] for t in range(len(Input))]
    x  = [[x[i][t] for i in range(len(x))] for t in range(A.shape[0])]
    return (y, U, x)

def simulation_ideal(sys_G, sys_K, sys_int):
    K_temp = control.append(sys_K,sys_int)
    Ka = sss(K_temp.A, 
            K_temp.B*sym.Matrix([[1,0,0],[0,1,0],[0,1,0],[0,0,1]]), 
            sym.Matrix([1,1]).T*K_temp.C, 
            sym.Matrix([1,1]).T*K_temp.D*sym.Matrix([[1,0,0],[0,1,0],[0,1,0],[0,0,1]]))
    sys_temp = control.append(sys_G,Ka)
    sys_temp1 = sss(sys_temp.A, 
                    sys_temp.B*sym.Matrix([[1,0,0],[1,0,0],[0,1,0],[0,0,1]]),
                    sys_temp.C,
                    sys_temp.D*sym.Matrix([[1,0,0],[1,0,0],[0,1,0],[0,0,1]]))
    #sys = control.connect(sys_temp1,[[2,1],[1,-2]],[3],[1,2])
    sys = sss(sys_temp1.A+sys_temp1.B[:,1]*sys_temp1.C[0,:]-sys_temp1.B[:,0]*sys_temp1.C[1,:],
              sys_temp1.B[:,2],
              sys_temp1.C,
              [[0],[0]])
    return sys

In [6]:
sols = numpy.linalg.eig(A)
x0 = sym.Matrix([0,0])
lim = 1.

def main_callback(X0w, K, L, eig1c, eig2c, eig3c, eig1o, eig2o, u, period, selm, sele, selu, sim, time, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(Aa-Ba*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        if eige == 0:
            K = control.acker(Aa, Ba, [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig1o[0,0], eig2o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(Aa, Ba, [eig3c[0,0], 
                                     numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)

    # Simulation without saturation
    sys_G   = sss(A,B,C,0)
    sys_K   = sss(A-L*C,sym.Matrix(B).row_join(sym.Matrix(L)),K[0][0,:2],[0,0])
    sys_int = sss(0,sym.Matrix([1,-1]).T,K[0][0,2],[0,0])
    x0 = sym.Matrix([0.,0.])
    sys = simulation_ideal(sys_G, sys_K, sys_int)
    solc = sym.Matrix(sys.A).eigenvals(multiple=True,rational=False)
    solc = numpy.matrix(solc,dtype=numpy.complex128)
    print('A rendszer sajátértékei:', round(sols[0][0],2),'és', round(sols[0][1],2))
    print('A szabályozott zárt hurok sajátértékei:', 
          round(solc[0,0],2),',', 
          round(solc[0,1],2),',', 
          round(solc[0,2],2),',',
          round(solc[0,3],2),'és',
          round(solc[0,4],2))
    
    X0w1 = [float(x0[0,0]),float(x0[1,0]),float(X0w[0,0]),float(X0w[1,0]),0.]
    T, Td = numpy.linspace(0,time,1000,retstep=True)
      
    if selu == 'impulzus': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        if sim == False:
            T, yout, xout = control.forced_response(sys,T,U,X0w1)
    if selu == 'egységugrás':
        U = [u for t in range(0,len(T))]
        if sim == False:
            T, yout, xout = control.forced_response(sys,T,U,X0w1)
    if selu == 'szinusz':
        U = u*numpy.sin(2*numpy.pi/period*T)+u
        if sim == False:
            T, yout, xout = control.forced_response(sys,T,U,X0w1)
    if selu == 'négyszögjel':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))+u
        if sim == False:
            T, yout, xout = control.forced_response(sys,T,U,X0w1)
    
    if sim:
        y_sat, U_sat, x_sat = simulation(sys_G, sys_K, sys_int, x0, sym.Matrix(X0w), T, Td, U, lim)
    
    fig = plt.figure(num='Szimuláció', figsize=(14,12))
    fig.add_subplot(224)
    plt.title('Kimenet válasza')
    plt.ylabel('Kimenet')
    if sim:
        plt.plot(T,y_sat,T,U,'r--')
        plt.legend(['$y_{tel}$','Referencia'])
    else:
        plt.plot(T,yout[0],T,U,'r--')
        plt.legend(['$y$','Referencia'])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(221)
    plt.title('Az első állapot válasza')
    plt.ylabel('$x_1$')
    if sim:
        plt.plot(T,x_sat[0],T,x_sat[2])
        plt.legend(['$x_{1tel}$','$x_{1becs}$'])
    else:
        plt.plot(T,xout[0],T,xout[2])
        plt.legend(['$x_{1}$','$x_{1becs}$'])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Második állapot válasza')
    plt.ylabel('$x_2$')
    if sim:
        plt.plot(T,x_sat[1],T,x_sat[3])
        plt.legend(['$x_{2tel}$','$x_{2becs}$'])
    else:
        plt.plot(T,xout[1],T,xout[3])
        plt.legend(['$x_{2}$','$x_{2becs}$'])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Bemenet')
    plt.ylabel('Bemenet')
    if sim:
        plt.plot(T[:-1],U_sat[:-1],T,[1 for t in range(len(T))],'r--')
        plt.legend(['$Nyomaték$','$határ$'])
    else:
        plt.plot(T,-yout[1],T,[1 for t in range(len(T))],'r--')
        plt.legend(['$Nyomaték$','$határ$'])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    if selu == 'egységugrás':
        if sim:
            print('A maximális túllövés:',round((max(y_sat)-U[0])/U[0]*100,3),'százalék')
        else:
            print('A maximális túllövés:',round((max(yout[0])-U[0])/U[0]*100,3),'százalék')

   
alltogether = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Sajátértékek:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 becs.:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Sajátértékek:',border=3), 
                                          eig1o, 
                                          eig2o, 
                                          widgets.Label(' ',border=3),
                                          time]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period,
                                          widgets.Label(' ',border=3),widgets.Label(' ',border=3),sim]),
                            widgets.HBox([widgets.Label('',border=3),
                            START])])
out = widgets.interactive_output(main_callback, {'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 
                                                 'eig1o':eig1o, 'eig2o':eig2o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu,
                                                 'sim':sim, 'time':time, 'DW':DW})
out.layout.height = '840px'
display(out, alltogether)

Output(layout=Layout(height='840px'))